# Dx29 PhenSimilarty microservice old_version

## 1. Calcule phenSimilarity

This method allows to compare a list of symptoms with another

The input will have the following format:

```
    {
        "list_reference":[<list Symptoms IDs>],
        "list_compare":[<list Symptoms IDs>]
    }
 ```
The result will be a json with list_reference symptoms as keys and "others" key. 
For each symptom reference, the values will be json objects with the list_compare symptoms Ids that has any similarity with the first.

An example could be:

```json
HTTP/1.1 200 OK
{
  "HP:0001250": {
      "HP:0001250": {
         "Similarity": "Equal",
          "Score": 100
     }
  },
  "HP:0012638": {
      "HP:0001250": {
           "Similarity": "Successor",
           "Score": 50
       },
       "HP:0000118": {
          "Similarity": "Predecessor",
         "Score": 50
     }
  },
  "HP:0009115": {},
  "others": {
      "HP:0040064": {
          "Similarity": "None",
          "Score": 0
      }
   }
}
```
The POST request for execute the main functionality of the microservice will be: 
    **“http://localhost:8080/api/calculate”**
    
This service is deployed in Kubernetes, so requests can be made to: http://dx29dev.northeurope.cloudapp.azure.com/:
In this case, the POST request for execute the main functionality of the microservice will be: 
    **“http://dx29dev-api.northeurope.cloudapp.azure.com/api/v1/PhenSimilarity/calculate”**

### Success response

#### Success response - `Success 200`

| Name     | Type       | Description                           |
|----------|------------|---------------------------------------|
| Json | `Object` | <p>A dictionary with the reference symptoms and the information about the comparision with the list to compare.</p> |

#### Success response example

##### Success response example - `Success-Response:`

```json
HTTP/1.1 200 OK
{
  "HP:0001250": {
      "HP:0001250": {
         "Similarity": "Equal",
          "Score": 100
     }
  },
  "HP:0012638": {
      "HP:0001250": {
           "Similarity": "Successor",
           "Score": 50
       },
       "HP:0000118": {
          "Similarity": "Predecessor",
         "Score": 50
     }
  },
  "HP:0009115": {},
  "others": {
      "HP:0040064": {
          "Similarity": "None",
          "Score": 0
      }
   }
}
```

### Error response

#### Error response - `Error 4xx`

| Name     | Type       | Description                           |
|----------|------------|---------------------------------------|
| Json |  | <p>BodyFormat Body must be a json with list_reference and list_compare keys, and both keys must have array values</p> |

#### Error response example

##### Error response example - `Error-Response: Request with not correct body: null`

```json
HTTP/1.1 400 Bad Request
    Invalid Request data: SyntaxError: Unexpected token n in JSON at position 0
```

##### Error response example - `Error-Response: Request with not correct body format: send json instead of a list`

```json
HTTP/1.1 400 Bad Request
     {
         "message": [
             "You must provide list_reference item with a list of symptoms",
             "You must provide list_compare item with a list of symptoms"
         ]
     }
```


## 2. Testing the microservice

In [1]:
input_payload = {
    "list_reference": [
        "HP:0001250",
        "HP:0012638",
        "HP:0009115"
    ],
    "list_compare": [
        "HP:0001250",
        "HP:0000118",
        "HP:0040064"
    ]
}

In [2]:
import requests
import texttable as tt

r = requests.post("http://dx29dev-api.northeurope.cloudapp.azure.com/api/v1/PhenSimilarity/calculate",headers = {'Content-Type': 'application/json'}, json=input_payload)
# Save the result in JSON format
response = r.json()
# Create a table to represent the results and add format
results_table = tt.Texttable()

headings = ["Symptom Reference","Symptom Compare","Similarity","Score"]
results_table.header(headings)

results_table.set_cols_dtype(['t','t','t','t']) 
results_table.set_cols_align(["l", "l", "l", "l"])
results_table.set_cols_width(["20","20","20","20"])

# Get info: disease id, score Dx29, score genes, score symptoms, matches symptoms and matches genes

for symptomReference in list(response.keys()):
    
    symptomReference_firstTime = 1;
    result_SymptomReference = symptomReference
    
    result_listCompareValues = []
    
    for symptomCompare in response[symptomReference]:
        result_symptomCompare = symptomCompare
        result_similarity = response[symptomReference][symptomCompare]['Similarity']
        result_score = response[symptomReference][symptomCompare]['Score']
        result_listCompareValues.append([result_symptomCompare,result_similarity,result_score])
        
    if (len(result_listCompareValues) > 0 ):
        for item in result_listCompareValues:
            result_listReference = []
            if (symptomReference_firstTime == 1):
                symptomReference_firstTime = 0
                result_listReference = [result_SymptomReference] 
            else:
                result_listReference = [" "]
            result_full = []
            result_full = result_listReference
            result_full.extend(item)
            results_table.add_row(result_full)
            
    else:
        result_full = [result_SymptomReference,"-","-","-"]
        results_table.add_row(result_full)

# Draw the table & print
results_table_draw = results_table.draw();
print(results_table_draw)
print("\n")

+----------------------+----------------------+----------------------+----------------------+
|  Symptom Reference   |   Symptom Compare    |      Similarity      |        Score         |
+======================+======================+======================+======================+
| HP:0001250           | HP:0001250           | Equal                | 100                  |
+----------------------+----------------------+----------------------+----------------------+
| HP:0012638           | HP:0001250           | Successor            | 50                   |
+----------------------+----------------------+----------------------+----------------------+
|                      | HP:0000118           | Predecessor          | 50                   |
+----------------------+----------------------+----------------------+----------------------+
| HP:0009115           | -                    | -                    | -                    |
+----------------------+----------------------+-------------